# Importing Libarries

In [1]:
# !pip install gradio
# !pip install soundfile
# !pip install modelscope -U
# !pip install librosa
# !pip install soundfile
# !pip install webrtcvad
# !pip install git+https://github.com/openai/whisper.git 
# !pip install yt-dlp
# !pip install moviepy --upgrade

import os
# Update package lists and install ImageMagick
!apt-get update
!apt install imagemagick -y

# Remove line 88 of policy.xml to allow MoviePy to run properly
!sed -i '88d' /etc/ImageMagick-6/policy.xml 

# Create experiments directory
# !mkdir experiments
!mkdir ../audio_chunks
# Exit the script with code 0
# os._exit(0)


zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

sed: 1: "/etc/ImageMagick-6/poli ...": invalid command code m
mkdir: ../audio_chunks: File exists


In [2]:
# !rm -rf ../audio_chunks/*

# Code

In [3]:
import sys
import os   
from dotenv import load_dotenv
import glob

PARENT_DIR = os.path.dirname(os.getcwd())
SRC_DIR = os.path.join(PARENT_DIR, 'src')
sys.path.append(SRC_DIR)

from Processing import processing
from Audio_webrtc import Audio_webrtc
from Audio_speechbrain import Audio_speechbrain
from NoiseEliminator import NoiseEliminator
from Graph import graph

/Users/shotomorisaki/Engineering/video-pipeline/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures
2024-03-03 10:21:13,950 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-03-03 10:21:13,952 - modelscope - INFO - Loading ast index from /Users/shotomorisaki/.cache/modelscope/ast_indexer
2024-03-03 10:21:13,994 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 355d5799ece8f8308a56fa0bf4e81920 and a total number of 964 components indexed


In [4]:
data_path = glob.glob(os.path.join(PARENT_DIR, 'data', '*'))
print(data_path)

['/Users/shotomorisaki/Engineering/video-pipeline/data/jacob.m4a', '/Users/shotomorisaki/Engineering/video-pipeline/data/test.m4a', '/Users/shotomorisaki/Engineering/video-pipeline/data/input.m4a', '/Users/shotomorisaki/Engineering/video-pipeline/data/output.wav']


In [9]:

# Example usage
if __name__ == "__main__":
	splitter = Audio_webrtc(
		top_db=20,
	)
	eliminator = NoiseEliminator()
	audio_file_path = data_path[0]
	output_path = "../audio_chunks"

	chunks = splitter.process_audio(audio_file_path, output_path)
	
	for chunk in chunks:
		eliminator.noise_suppression(chunk, chunk)


2024-03-03 10:29:27,829 - modelscope - WARNING - Model revision not specified, use revision: v1.0.2
2024-03-03 10:29:28,268 - modelscope - INFO - initiate model from /Users/shotomorisaki/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k
2024-03-03 10:29:28,268 - modelscope - INFO - initiate model from location /Users/shotomorisaki/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k.
2024-03-03 10:29:28,270 - modelscope - INFO - initialize model from /Users/shotomorisaki/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k
2024-03-03 10:29:28,911 - modelscope - WARNING - No preprocessor field found in cfg.
2024-03-03 10:29:28,912 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-03-03 10:29:28,912 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/Users/shotomorisaki/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k'}. trying to build by task an

Audio File: /Users/shotomorisaki/Engineering/video-pipeline/data/jacob.m4a
inputs:(1, 506368)
padding: 26368
inputs after padding:(1, 532736)
inputs:(1, 507904)
padding: 27904
inputs after padding:(1, 535808)
inputs:(1, 485888)
padding: 17888
inputs after padding:(1, 503776)
inputs:(1, 488960)
padding: 20960
inputs after padding:(1, 509920)
inputs:(1, 492032)
padding: 24032
inputs after padding:(1, 516064)
inputs:(1, 501760)
padding: 21760
inputs after padding:(1, 523520)
inputs:(1, 504320)
padding: 24320
inputs after padding:(1, 528640)


In [18]:
from pydub import AudioSegment

combined_audio_path = os.path.join(output_path, "combined_audio.wav")
combined_audio = AudioSegment.empty()
for chunk in chunks:
    combined_audio += AudioSegment.from_file(chunk)

# Save the combined audio
combined_audio.export(combined_audio_path, format='wav')

<_io.BufferedRandom name='../audio_chunks/combined_audio.wav'>

In [17]:
files = glob.glob(os.path.join(PARENT_DIR, 'audio_chunks', '*'))

IsADirectoryError: [Errno 21] Is a directory: '/'

In [14]:
# graph().audio_analysis(files[1])

IsADirectoryError: [Errno 21] Is a directory: '/'

In [ ]:
print(chunks)

In [ ]:
print(a)

import glob

chunks = sorted(glob.glob("/notebooks/audio_chunks/*.wav"))
chunks

In [ ]:
import os
import glob
import json
import whisper
import librosa
from tqdm import tqdm

class TranscriptionProcessor:
    def __init__(self, model_type='medium', output_json_path="/notebooks/transcriptions.json"):
        self.model = whisper.load_model(model_type).cuda()
        self.output_json_path = output_json_path
        self.segments = []
        self.cumulative_duration = 0
    
    def transcribe_chunks(self, chunks):
        if os.path.exists(self.output_json_path):
            os.remove(self.output_json_path)
        
        for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
            transcription = self.model.transcribe(chunk)
            self._update_segments(transcription["segments"])
            self._save_transcription()
            self.cumulative_duration += librosa.get_duration(filename=chunk)
    
    def _update_segments(self, new_segments):
        for segment in new_segments:
            segment["start"] += self.cumulative_duration
            segment["end"] += self.cumulative_duration
        self.segments.extend(new_segments)
    
    def _save_transcription(self):
        with open(self.output_json_path, "w") as file:
            json.dump(self.segments, file, indent=2)

# Example usage
if __name__ == "__main__":
    chunks = sorted(glob.glob("/notebooks/audio_chunks/*.wav"))
    processor = TranscriptionProcessor()
    processor.transcribe_chunks(chunks)


In [ ]:
import subprocess
import json
import os
import librosa

def concatenate_audio(chunks, output_file):
	"""Concatenate a list of audio chunks into a single file."""
	with open('concat_list.txt', 'w') as f:
		for chunk in chunks:
			f.write(f"file '{chunk}'\n")
	subprocess.run(['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'concat_list.txt', 
					'-c', 'copy', output_file], check=True)
	os.remove('concat_list.txt')

def generate_srt_from_json(segments, srt_file):
	"""Generate an SRT file from the transcription JSON, considering each chunk's cumulative time."""
	with open(srt_file, 'w', encoding='utf-8') as file:
		for entry in segments:
			# Adjust start and end times by adding the cumulative duration
			start = entry['start']
			end = entry['end']
			text = entry['text']
			file.write(f"{entry['id'] + 1}\n")
			file.write(f"{format_time(start)} --> {format_time(end)}\n")
			file.write(f"{text}\n\n")
			
def format_time(seconds):
	"""Convert time in seconds to SRT time format."""
	hours = int(seconds // 3600)
	minutes = int((seconds % 3600) // 60)
	seconds = seconds % 60
	return f"{hours:02}:{minutes:02}:{seconds:06.3f}".replace('.', ',')

def create_video_with_subtitles_from_json(image_path, json_path, chunks, output_video_path):
	# Read JSON to get all audio paths
	with open(json_path, 'r') as file:
		segments = json.load(file)

	combined_audio_path = 'combined_audio.wav'
	concatenate_audio(chunks, combined_audio_path)

	srt_file = 'subtitles.srt'
	generate_srt_from_json(segments, srt_file)

	if os.path.exists(output_video_path):
		os.remove(output_video_path)
	
	# Command to create video using FFmpeg
	command = [
		'ffmpeg',
		'-loop', '1',                 # Loop the image
		'-i', image_path,             # Input image file
		'-i', combined_audio_path,    # Input combined audio file
		'-vf', "scale=1240:814,subtitles=" + srt_file,
		'-r', '1',                    # Set output frame rate to 1 fps
		'-c:v', 'libx264',            # Video codec
		'-pix_fmt', 'yuv420p',        # Pixel format
		'-tune', 'stillimage',        # Tune for still image
		'-c:a', 'aac',                # Audio codec
		'-b:a', '192k',               # Audio bitrate
		'-shortest',                  # Finish encoding when the shortest stream ends
		output_video_path             # Output file
	]

	subprocess.run(command, check=True)
	# Clean up
	os.remove(combined_audio_path)
	os.remove(srt_file)

# Example usage
background_image_path = '/notebooks/Background 1240x815.jpg'
json_path = '/notebooks/transcriptions.json'
chunks = sorted(glob.glob("/notebooks/audio_chunks/*.wav"))
output_video_path = 'output_video.mp4'

create_video_with_subtitles_from_json(background_image_path, json_path, chunks, output_video_path)
